In [8]:
import os, sys

project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
os.environ['PYTHONPATH'] = project_root
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print("PYTHONPATH manually set to:", os.environ['PYTHONPATH'])

PYTHONPATH manually set to: c:\Users\ndhaj\Desktop


## Holography

In [13]:
import numpy as np
from gpie import Graph, SupportPrior, ConstWave, fft2, AmplitudeMeasurement, mse
from gpie.core.linalg_utils import circular_aperture, masked_random_array

# ==== Setting Parameter ====
H, W = 128, 128
shape = (H, W)
var = 1e-4
rng = np.random.default_rng(seed=42)
support_x = circular_aperture(shape=shape, radius=0.2, center=(-0.2, -0.2))
data_x = masked_random_array(support_x, dtype=np.complex128, rng=rng)
support_y =  circular_aperture(shape=shape, radius=0.2, center=(0.2, 0.2))

# ==== Graph ====
class HolographicCDI(Graph):
    def __init__(self):
        super().__init__()
        # known and unknown wave
        Y = ~SupportPrior(support = support_y, label="Y")
        # diffraction measurement
        with self.observe():
            meas = AmplitudeMeasurement(var = var, damping = 0.0) @ (fft2(data_x + Y))
        self.compile()

# ==== グラフ構築・サンプル生成 ====
g = HolographicCDI()
g.set_init_rng(np.random.default_rng(seed=11))
g.generate_sample(rng=np.random.default_rng(seed=9), update_observed=True)

true_y = g.get_wave("Y").get_sample()

# ==== 推論ループ ====
def monitor(graph, t):
    y = graph.get_wave("Y").compute_belief().data
    err = mse(y, true_y)
    if t % 100 == 0:
        print(f"[t={t}] PMSE (sum) = {err:.5e}")

g.run(n_iter=1000, callback=monitor, verbose = False)


[t=0] PMSE (sum) = 4.20747e-01
[t=100] PMSE (sum) = 3.71758e-04
[t=200] PMSE (sum) = 3.71758e-04
[t=300] PMSE (sum) = 3.71758e-04
[t=400] PMSE (sum) = 3.71758e-04
[t=500] PMSE (sum) = 3.71758e-04
[t=600] PMSE (sum) = 3.71758e-04
[t=700] PMSE (sum) = 3.71758e-04
[t=800] PMSE (sum) = 3.71758e-04
[t=900] PMSE (sum) = 3.71758e-04


In [10]:
g.visualize()

Loading BokehJS ...